In [1]:
#!/usr/bin/env python
#coding:utf-8

"""
Each sudoku board is represented as a dictionary with string keys and
int values.
e.g. my_board['A1'] = 8
"""
import sys

ROW = "ABCDEFGHI"
COL = "123456789"

In [2]:
def print_board(board):
    """Helper function to print board in a square."""
    print("-----------------")
    for i in ROW:
        row = ''
        for j in COL:
            row += (str(board[i + j]) + " ")
        print(row)

In [3]:
def board_to_string(board):
    """Helper function to convert board dictionary to string for writing."""
    ordered_vals = []
    for r in ROW:
        for c in COL:
            ordered_vals.append(str(board[r + c]))
    return ''.join(ordered_vals)

In [26]:
def get_row(var):
    row = var[0]
    rows = []
    for i in range(9):
        r = chr(ord("1") + i)
        rows.append(row+r)
    return rows

In [24]:
def get_col(var):
    col = var[1]
    cols = []
    for i in range(9):
        c = chr(ord("A") + i)
        cols.append(c+col)
    return cols

In [20]:
def get_mini_grid(var):
    mini_grids = {
        "00" : ['A1','A2','A3','B1','B2','B3','C1','C2','C3'],
        "01" : ['A4','A5','A6','B4','B5','B6','C4','C5','C6'],
        "02" : ['A7','A8','A9','B7','B8','B9','C7','C8','C9'],
        "10" : ['D1','D2','D3','E1','E2','E3','F1','F2','F3'],
        "11" : ['D4','D5','D6','E4','E5','E6','F4','F5','F6'],
        "12" : ['D7','D8','D9','E7','E8','E9','F7','F8','F9'],
        "20" : ['G1','G2','G3','H1','H2','H3','I1','I2','I3'],
        "21" : ['G4','G5','G6','H4','H5','H6','I4','I5','I6'],
        "22" : ['G7','G8','G9','H7','H8','H9','I7','I8','I9']}
    row = (ord(var[0]) - ord('A')) // 3
    col = (ord(var[1]) - ord('1')) // 3
    return mini_grids[str(row) + str(col)]

In [29]:
def satisfy_constraints(board):
    return row_constraint(board) and col_constraint(board) and mini_grid_constraint(board)

In [135]:
def get_neighbors(var):
    row_n = set(get_row(var))
    col_n = set(get_col(var))
    mini_grid_n = set(get_mini_grid(var))
    n = row_n | col_n | mini_grid_n
    n.remove(var)
    return n

In [127]:
def all_tiles_filled(board):
    board_str = board_to_string(board)
    return not ("0" in board_str)

In [137]:
def select_var_mrv(assignment, domains):
    min_domain_len = 100
    var = None

    for k,v in assignment.items():
        if(v == 0 and len(domains[k]) < min_domain_len):
            min_domain_len = len(domains[k])
            var = k

    return var

In [138]:
def is_safe(value, var, assignment, neighbors):
    neighbor = neighbors[var]
    for n in neighbor:
        if (assignment[n] == value):
            return False

    return True

In [299]:
def forward_checking(assignment, neighbors, domains, var):
    new_domains = dict(domains)
    
    for neighbor in neighbors[var]:
        if assignment[var] in new_domains[neighbor]:
            new_domains[neighbor].remove(assignment[var])
            if (len(new_domains[neighbor]) == 0):
                return False, domains

    return True, new_domains

In [300]:
def backtrack(assignment, neighbors, domains):
    if (all_tiles_filled(assignment)):
        return True

    var = select_var_mrv(assignment, domains)
    domain = list(domains[var])
    domains_cp = dict(domains)
    
    for v in domain:
        if(var == "A1"):
            print(v, domain, domains[var], domains_cp[var])
            
        if (v not in domains[var]):
            continue

        if (is_safe(v, var, assignment, neighbors)):
            assignment[var] = v

            FC_success, new_domains = forward_checking(assignment, neighbors, domains, var)

            if (FC_success):
                backtrack_success = backtrack(assignment, neighbors, new_domains)
                print(backtrack_success, new_domains["A1"], domains["A1"], domains_cp["A1"])
                if (backtrack_success):
                    return True
                else:
                    domains = domains_cp
            else:
                domains = domains_cp
                assignment[var] = 0
    
    return False

In [301]:
def backtracking(board):
    """Takes a board and returns solved board."""    
    assignment = dict(board)
    
    neighbors = dict(board)
    for k in neighbors:
        neighbors[k] = get_neighbors(k)

    domains = dict(board)
    for k in domains:
        domains[k] = {1,2,3,4,5,6,7,8,9}

    # forward checking beforehand
    for var,v in board.items():
        forward_checking(assignment, neighbors, domains, var)

    # for item in domains.items():
    #     print(item)

    state = backtrack(assignment, neighbors, domains)

    print_board(assignment)

    if (state):
        return assignment
    else:
        return "Failed to solve this board"

In [302]:
if __name__ == '__main__':
    input_str = "000100702030950000001002003590000301020000070703000098800200100000085060605009000"
    board = { ROW[r] + COL[c]: int(input_str[9*r+c])
                  for r in range(9) for c in range(9)}       

    print_board(board)
    solution = backtracking(board)
    print_board(solution)

-----------------
0 0 0 1 0 0 7 0 2 
0 3 0 9 5 0 0 0 0 
0 0 1 0 0 2 0 0 3 
5 9 0 0 0 0 3 0 1 
0 2 0 0 0 0 0 7 0 
7 0 3 0 0 0 0 9 8 
8 0 0 2 0 0 1 0 0 
0 0 0 0 8 5 0 6 0 
6 0 5 0 0 9 0 0 0 
4 [4, 9] {4, 9} {4, 9}
False {4} {4} {4}
False {4} {4} {4}
False {4} {4} {4}
False {4} {4} {4}
False {4} {4} {4}
False {4} {4} {4}
False {4} {4} {4}
False {4} {4} {4}
9 [4, 9] {4} {4}
-----------------
4 0 6 1 3 8 7 0 2 
2 3 0 9 5 0 0 0 0 
9 0 1 0 0 2 0 0 3 
5 9 0 0 0 0 3 0 1 
1 2 0 0 0 0 0 7 0 
7 0 3 0 0 0 0 9 8 
8 0 0 2 0 0 1 0 0 
3 0 0 0 8 5 0 6 0 
6 0 5 0 0 9 0 0 0 
-----------------


TypeError: string indices must be integers, not 'str'

In [303]:
if __name__ == '__main__':
    if len(sys.argv) > 1:
        
        # Running sudoku solver with one board $python3 sudoku.py <input_string>.
        print(sys.argv[1])
        # Parse boards to dict representation, scanning board L to R, Up to Down
        board = { ROW[r] + COL[c]: int(sys.argv[1][9*r+c])
                  for r in range(9) for c in range(9)}       
        
        solved_board = backtracking(board)
        
        # Write board to file
        out_filename = 'output.txt'
        outfile = open(out_filename, "w")
        outfile.write(board_to_string(solved_board))
        outfile.write('\n')

    else:
        # Running sudoku solver for boards in sudokus_start.txt $python3 sudoku.py

        #  Read boards from source.
        src_filename = 'sudokus_start.txt'
        try:
            srcfile = open(src_filename, "r")
            sudoku_list = srcfile.read()
        except:
            print("Error reading the sudoku file %s" % src_filename)
            exit()

        # Setup output file
        out_filename = 'output.txt'
        outfile = open(out_filename, "w")

        # Solve each board using backtracking
        for line in sudoku_list.split("\n"):

            if len(line) < 9:
                continue

            # Parse boards to dict representation, scanning board L to R, Up to Down
            board = { ROW[r] + COL[c]: int(line[9*r+c])
                      for r in range(9) for c in range(9)}

            # Print starting board. TODO: Comment this out when timing runs.
            print_board(board)

            # Solve with backtracking
            solved_board = backtracking(board)

            # Print solved board. TODO: Comment this out when timing runs.
            print_board(solved_board)

            # Write board to file
            outfile.write(board_to_string(solved_board))
            outfile.write('\n')

        print("Finishing all boards in file.")

-f


ValueError: invalid literal for int() with base 10: '-'